In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
train_path="C:/Users/user/Desktop/Uni/Viti 3/Semestri 1/Artificial Intelligence/project_dataset/train"
test_path="C:/Users/user/Desktop/Uni/Viti 3/Semestri 1/Artificial Intelligence/project_dataset/test"
val_path="C:/Users/user/Desktop/Uni/Viti 3/Semestri 1/Artificial Intelligence/project_dataset/val"

In [3]:
img_height = 100
img_width = 100

x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(img_height,img_width))

        x_train.append(img_arr)

x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(img_height,img_width))

        x_test.append(img_arr)

x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(img_height,img_width))

        x_val.append(img_arr)

In [4]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [6]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (img_height,img_width),
                                                 batch_size = 8,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (img_height,img_width),
                                            batch_size = 8,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (img_height,img_width),
                                            batch_size = 8,
                                            class_mode = 'sparse')

Found 4432 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [7]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [8]:
training_set.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [9]:
train_y.shape,test_y.shape,val_y.shape

((4432,), (624,), (800,))

In [10]:
model = Sequential([ 
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(2, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),  
  layers.Conv2D(4, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dense(2)
])

In [11]:
model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']
)
    
model.build()

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 100, 100, 2)       56        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 2)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 4)         76        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 4)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 8)         2

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [14]:
epochs=5
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=epochs,
  callbacks=[early_stop],
  batch_size=8,
  shuffle=True,
)

Epoch 1/5
554/554 [==============================] - 76s 135ms/step - loss: 0.6935 - accuracy: 0.2155 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/5
554/554 [==============================] - 72s 129ms/step - loss: 0.6931 - accuracy: 0.2141 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/5
554/554 [==============================] - 72s 129ms/step - loss: 0.6931 - accuracy: 0.2141 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/5
554/554 [==============================] - 72s 129ms/step - loss: 0.6931 - accuracy: 0.2141 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/5
554/554 [==============================] - 68s 123ms/step - loss: 0.6931 - accuracy: 0.2141 - val_loss: 0.6931 - val_accuracy: 0.5000


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [19]:
model.evaluate(test_x,test_y,batch_size=32)

20/20 [==============================] - 1s 8ms/step - loss: 0.6931 - accuracy: 0.6250


[0.6931473612785339, 0.625]

In [20]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import numpy as np

In [21]:
#predict
y_pred=model.predict(test_x)
y_pred=np.argmax(y_pred,axis=1)

20/20 [==============================] - 0s 7ms/step


In [22]:
#get classification report
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.62      0.77       624

    accuracy                           0.62       624
   macro avg       0.50      0.31      0.38       624
weighted avg       1.00      0.62      0.77       624



C:\Users\user\anaconda3\envs\proect_work\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\envs\proect_work\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\user\anaconda3\envs\proect_work\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
#get confusion matrix
print(confusion_matrix(y_pred,test_y))

[[  0   0]
 [234 390]]
